# Lab 5: Agentic Rag and External Memory

## Preparation

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>
</div>

## Section 0: Setup a Letta client

In [1]:
from letta_client import Letta

client = Letta(base_url="http://localhost:8283")

In [2]:
def print_message(message):  
    if message.message_type == "reasoning_message": 
        print("🧠 Reasoning: " + message.reasoning) 
    elif message.message_type == "assistant_message": 
        print("🤖 Agent: " + message.content) 
    elif message.message_type == "tool_call_message": 
        print("🔧 Tool Call: " + message.tool_call.name + "\n" + message.tool_call.arguments)
    elif message.message_type == "tool_return_message": 
        print("🔧 Tool Return: " + message.tool_return)
    elif message.message_type == "user_message": 
        print("👤 User Message: " + message.content)

## Section 1: Data Sources

### Creating a source

In [3]:
source = client.sources.create(
    name="employee_handbook",
    embedding="openai/text-embedding-3-small"
)
source

Source(id='source-5904362f-8639-44eb-8eec-44c163c2f619', name='employee_handbook', description=None, embedding_config=EmbeddingConfig(embedding_endpoint_type='openai', embedding_endpoint='http://jupyter-api-proxy.internal.dlai/rev-proxy/letta', embedding_model='text-embedding-3-small', embedding_dim=2000, embedding_chunk_size=300, handle='openai/text-embedding-3-small', azure_endpoint=None, azure_version=None, azure_deployment=None), metadata=None, created_by_id='user-00000000-0000-4000-8000-000000000000', last_updated_by_id='user-00000000-0000-4000-8000-000000000000', created_at=datetime.datetime(2025, 8, 16, 3, 45, 36), updated_at=datetime.datetime(2025, 8, 16, 3, 45, 36), organization_id='org-00000000-0000-4000-8000-000000000000')

### Uploading a source

In [4]:
job = client.sources.files.upload(
    source_id=source.id,
    file=open("handbook.pdf", "rb")
)

In [5]:
job.status

'created'

### Viewing job status over time

In [6]:
import time
from letta_client import JobStatus

while job.status != 'completed':
    job = client.jobs.retrieve(job.id)
    print(job.status)
    time.sleep(1)

completed


### Viewing job metadata

In [7]:
job.metadata

{'type': 'embedding',
 'filename': 'handbook.pdf',
 'source_id': 'source-5904362f-8639-44eb-8eec-44c163c2f619',
 'num_passages': 11,
 'num_documents': 1}

In [8]:
passages = client.sources.passages.list(
    source_id=source.id,
)
len(passages)

11

### Creating an agent and attaching sources

In [9]:
agent_state = client.agents.create(
    memory_blocks=[
        {
          "label": "human",
          "value": "My name is Sarah"
        },
        {
          "label": "persona",
          "value": "You are a helpful assistant"
        }
    ],
    model="openai/gpt-4o-mini-2024-07-18",
    embedding="openai/text-embedding-3-small"
)

In [10]:
agent_state = client.agents.sources.attach(
    agent_id=agent_state.id, 
    source_id=source.id
)

### Viewing agent's attached sources

In [11]:
client.agents.sources.list(agent_id=agent_state.id)

[Source(id='source-5904362f-8639-44eb-8eec-44c163c2f619', name='employee_handbook', description=None, embedding_config=EmbeddingConfig(embedding_endpoint_type='openai', embedding_endpoint='http://jupyter-api-proxy.internal.dlai/rev-proxy/letta', embedding_model='text-embedding-3-small', embedding_dim=2000, embedding_chunk_size=300, handle='openai/text-embedding-3-small', azure_endpoint=None, azure_version=None, azure_deployment=None), metadata=None, created_by_id='user-00000000-0000-4000-8000-000000000000', last_updated_by_id='user-00000000-0000-4000-8000-000000000000', created_at=datetime.datetime(2025, 8, 16, 3, 45, 36), updated_at=datetime.datetime(2025, 8, 16, 3, 45, 36), organization_id='org-00000000-0000-4000-8000-000000000000')]

In [12]:
passages = client.agents.passages.list(agent_id=agent_state.id)
len(passages)

11

In [14]:
passages[0]

Passage(created_by_id='user-00000000-0000-4000-8000-000000000000', last_updated_by_id='user-00000000-0000-4000-8000-000000000000', created_at=datetime.datetime(2025, 8, 16, 3, 45, 40, 344308), updated_at=datetime.datetime(2025, 8, 16, 3, 45, 40), is_deleted=False, agent_id=None, source_id='source-5904362f-8639-44eb-8eec-44c163c2f619', file_id='file-a2bb2a41-0d1f-4d87-90da-cf0930286bb2', metadata={}, id='passage-5cc70810-c375-4030-b937-8aeb8cd68c1f', text="EmployeeHandbook\nTableofContents\n1. Introduction2. CompanyMissionandValues3. EmploymentPolicies○ 3.1WorkingHours○ 3.2CompensationandBenefits○ 3.3PerformanceEvaluation4. CodeofConduct5. VacationPolicy6. ConfidentialityAgreement7. IntellectualProperty8. DisciplinaryProcedures9. Acknowledgment\n1.Introduction\nWelcometoClosedAI Corporation. Wearepleasedtohaveyoujoinourteamofdedicatedprofessionalscommittedtoadvancingthefrontiersof artificial intelligenceandmachinelearningtechnologies. Asaleadingentityinthisrapidlyevolvingindustry, wepri

### Messaging agents and referencing attached sources

In [15]:
response = client.agents.messages.create(
    agent_id=agent_state.id,
    messages=[
        {
            "role": "user",
            "content": "Search archival for our company's vacation policies"
        }
    ]
)
for message in response.messages:
    print_message(message)

🧠 Reasoning: User wants to search for vacation policies. Time to look into the archival memory for relevant information.
🔧 Tool Call: archival_memory_search
{
  "query": "vacation policies",
  "page": 0,
  "start": 0,
  "request_heartbeat": true
}
🔧 Tool Return: ([{'timestamp': '2025-08-16 03:45:40.433376', 'content': "priorities. Assuch, vacationsarepermittedonlyunderthefollowingcondition: youmust provideanAI agent that matchesorsurpassesyourowncompetenciestofullyperformyourdutiesduringyourabsence.\nTheAI replacement must beequivalentlycompetent inall aspectsof yourrole, ensuringseamlesscontinuityof operations. Youarerequiredtosubmit theAI agent toyourimmediatesupervisorat least fourweekspriortoyourintendedleavedate. Thistimeframeallowsforrigoroustestingandevaluationof theAI'scapabilitiesandreliability.\nTheAI will undergocomprehensiveassessmentstoverifyitsproficiencyandeffectivenessinhandlingyourresponsibilities. Approval of theAI agent isat thesolediscretionof uppermanagement, andsu

## Section 2: Connecting Data with Custom Tools

### Creating a custom tool

In [16]:
def query_birthday_db(name: str):
    """
    This tool queries an external database to
    lookup the birthday of someone given their name.

    Args:
        name (str): The name to look up

    Returns:
        birthday (str): The birthday in mm-dd-yyyy format

    """
    my_fake_data = {
        "bob": "03-06-1997",
        "sarah": "07-06-1993"
    }
    name = name.lower()
    if name not in my_fake_data:
        return None
    else:
        return my_fake_data[name]

In [17]:
birthday_tool = client.tools.upsert_from_function(func=query_birthday_db)

### Creating an agent with access to tools

In [18]:
agent_state = client.agents.create(
    memory_blocks=[
        {
          "label": "human",
          "value": "My name is Sarah"
        },
        {
          "label": "persona",
          "value": "You are a agent with access to a birthday_db " \
            + "that you use to lookup information about users' birthdays."
        }
    ],
    model="openai/gpt-4o-mini-2024-07-18",
    embedding="openai/text-embedding-3-small",
    tool_ids=[birthday_tool.id],
    #tool_exec_environment_variables={"DB_KEY": "my_key"}
)

In [19]:
# send a message to the agent
response = client.agents.messages.create_stream(
    agent_id=agent_state.id,
    messages=[
        {
            "role": "user",
            "content": "whens my bday????"
        }
    ]
)
for message in response:
    print_message(message)

🧠 Reasoning: User wants to know their birthday. Looking it up in the birthday database.
🔧 Tool Call: query_birthday_db
{
  "name": "Sarah",
  "request_heartbeat": true
}
🔧 Tool Return: 07-06-1993
🧠 Reasoning: Birthday retrieved successfully. Sending the information to Sarah.
🤖 Agent: Your birthday is July 6, 1993! 🎉
